# Pandas and Beyond, Scaling Python Up To Bigger Data and Faster Compute

## Use Pandas Where You Can, Scale Up Where You Must

In [ ]:
from IPython.display import Image

In [ ]:
Image("time_space.png")

## Pandas is talked about more on Stack Overflow than a lot of popular programming languages (currently talked about as much as C++)

In [ ]:
Image("pandas_on_so.png")

## Time to run can become excessive because either the CPU / GPU is crunching many numbers or because we're blocked waiting on some data or resource to become available

CPUs run much faster than human brains - depending on where you read data from the CPU has to wait vastly different amounts of time even though the time difference might be relatively small at the human scale.

### Analogy for comparing human and CPU time for IO bound tasks

| Human Time | CPU Time |
| - | - |
| Fetching papers on your desktop and reading them | Fetching data from the CPU cache memory |
| Searching your desk draw for a paper - put it on the desktop and reading it | Fetching data from RAM memory |
| Travel to your local library, pick up a book, return to your desk and read it | Fetching data from your SSD, HDD, or external disk drive |
| Travel to your local library, create inter-library loan request, leave, come back when the book arrives, return with it to your desk | Reading data from a database or API from another computer on the network or Internet |

### Quick Pandas Example - Download Australian Energy Market Operator (AEMO) Supervisory Control and Data Acquisition (SCADA) Data and join into a single table

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
date_range = pd.date_range(start="Sept 2006", end=pd.Timestamp.today(), freq="m")
csv_file_list = [
    f"""http://data.wa.aemo.com.au/datafiles/facility-scada/facility-scada-{date.strftime("%Y-%m")}.csv"""
    for date in date_range
]

In [25]:
%%time
facility_scada_df = pd.concat(
    [pd.read_csv(csv_file) for csv_file in tqdm(csv_file_list)], ignore_index=True
)

/home/owen/miniconda3/envs/jupyter/lib/python3.8/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)



CPU times: user 22.4 s, sys: 13.7 s, total: 36.1 s
Wall time: 20min 44s


In [26]:
facility_scada_df.to_parquet("facility_scada.snappy")

In [ ]:
facility_scada_df = pd.read_parquet("facility_scada.snappy")

In [ ]:
facility_scada_df.shape

In [ ]:
facility_scada_df.head()

## Show off how easily we can maniuplate and plot data with Pandas

In [ ]:
%%time
facility_scada_df = pd.read_parquet("facility_scada.snappy")
facility_scada_df["Participant Facility"] = facility_scada_df[
    "Participant Code"
].str.cat(facility_scada_df["Facility Code"], sep=" - ")

aemo_scada_pivot_df = facility_scada_df.pivot(
    index="Trading Interval",
    columns="Participant Facility",
    values="Energy Generated (MWh)",
)
aemo_scada_pivot_df.index = pd.to_datetime(aemo_scada_pivot_df.index)
aemo_scada_pivot_df /= 1000.0
top_10_generators = aemo_scada_pivot_df.sum().sort_values(ascending=False).index[:10]

In [ ]:
aemo_scada_pivot_df.shape

In [ ]:
aemo_scada_pivot_df.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%%time
with plt.style.context('seaborn-bright'):
    ax = aemo_scada_pivot_df.loc[:, top_10_generators].resample("D").sum().cumsum().plot(
        figsize=(12, 6),
        title="Energy Generated"
    )
    ax.set_xlabel("Time")
    ax.set_ylabel("Energy Generated (GWh)")

## Lets try downloading those CSV files again - but use all the CPU

In [10]:
from joblib import Parallel, delayed, cpu_count

In [11]:
cpu_count()

16

In [16]:
%%time
facility_scada_parallel_df = pd.concat(
    Parallel(n_jobs=-1)(
        delayed(pd.read_csv)(csv_file) for csv_file in tqdm(csv_file_list)
    ),
    ignore_index=True,
)


CPU times: user 2.44 s, sys: 1.15 s, total: 3.59 s
Wall time: 4min 38s


In [60]:
facility_scada_parallel_df.equals(facility_scada_df)

True

## Lets try downloading those CSV files again - but Asynchronously

In [21]:
import asyncio
import aiohttp
import io
import time

In [22]:
async def download_file(session: aiohttp.ClientSession, file_url: str):
    async with session.get(file_url) as resp:
        return pd.read_csv(io.BytesIO(await resp.read()))


async def download_file_list(file_url_list):
    async with aiohttp.ClientSession() as session:
        tasks = [download_file(session, file_url) for file_url in file_url_list]
        return await asyncio.gather(*tasks)

In [23]:
t0 = time.time()
facility_scada_async_df = pd.concat(await download_file_list(csv_file_list), ignore_index=True)
t1 = time.time()
print(f"Wall time: {t1 - t0:10.2f}s")

/home/owen/miniconda3/envs/jupyter/lib/python3.8/asyncio/base_events.py:570: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  self._run_once()


Wall time:      46.17s


In [61]:
facility_scada_async_df.equals(facility_scada_df)

True

## Have a look at the New York Taxi Dataset

You can download the individual NY Taxi CSV files [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

In [ ]:
%%time
ny_taxi_june_2019_df = pd.read_csv("yellow_tripdata_2019-06.csv")

In [ ]:
memory_usage_bytes = ny_taxi_june_2019_df.memory_usage().sum()
print(f"DataFrame memory usage (Mb): {memory_usage_bytes / 1024 / 1024}")

In [ ]:
%%time
ny_taxi_june_2019_df.shape

In [ ]:
%%time
ny_taxi_june_2019_df["trip_distance"].mean()

In [ ]:
%%timeit
ny_taxi_june_2019_df["trip_distance"].mean()

## Scaling up with CuDF (Linux only)

In [ ]:
import cudf

In [ ]:
%%time
ny_taxi_june_2019_cdf = cudf.read_csv("yellow_tripdata_2019-06.csv")

In [ ]:
memory_usage_bytes = ny_taxi_june_2019_cdf.memory_usage().sum()
print(f"DataFrame memory usage (Mb): {memory_usage_bytes / 1024 / 1024}")

In [ ]:
%%time
ny_taxi_june_2019_cdf.shape

In [ ]:
%%time
ny_taxi_june_2019_cdf["trip_distance"].mean()

In [ ]:
%%timeit
ny_taxi_june_2019_cdf["trip_distance"].mean()

## Scaling up with Vaex

In [ ]:
import vaex

You can download the follow dataset file [here](https://vaex.readthedocs.io/en/latest/datasets.html) but do it at work unless you have unlimited downloads because it is over 100 gigs. There's a smaller 1 year dataset file which is about 12 gigs there too.

In [ ]:
%%time
ny_tax_2009_2015_df = vaex.open('yellow_taxi_2009_2015_f32.hdf5')

In [ ]:
ny_tax_2009_2015_df.shape

In [ ]:
%%time
ny_tax_2009_2015_df.head()

In [ ]:
%%time
ny_tax_2009_2015_df["trip_distance"].mean()

In [ ]:
%%time
long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90

ax = ny_tax_2009_2015_df.plot(
    ny_tax_2009_2015_df["pickup_longitude"],
    ny_tax_2009_2015_df["pickup_latitude"],
    f="log1p",
    limits=[[long_min, long_max], [lat_min, lat_max]],
    show=True,
    shape=2048,
    figsize=(18, 12),
)

## Scale up even more with Dask

### Create Dask Client, Insert Code, Show Graph Visual

In [ ]:
import dask.dataframe as dd

In [ ]:
%%time
ny_taxi_df = dd.read_parquet("yellow_taxi_2009_2015_f32.parquet")

In [ ]:
ny_taxi_df.npartitions

In [ ]:
%%time
ny_taxi_df.head()

In [ ]:
%%time
ny_taxi_df["trip_distance"].mean().compute()

## Beware - Native Python Types are Flexible but Very Memory Expensive

In [ ]:
import sys
import numpy as np

### Because everything is an object in Python even simple types like integers carry a lot of admin baggage - one number uses 28 bytes of memory - this could be 1, 2, or 4 bytes in other languages

In [ ]:
sys.getsizeof(1)

### A Python list of a million integers

In [ ]:
a_python_list = [number for number in range(1,1_000_000)]

In [ ]:
size_of_million_integers_in_list = sys.getsizeof(a_python_list) + len(a_python_list) * sys.getsizeof(a_python_list[0])
print(f"A Python list of a million integers in a list takes {size_of_million_integers_in_list / 1024 / 1024:5.2f} Mb")

### A Numpy array with the same integers

In [ ]:
a_numpy_array = np.arange(1,1_000_000)

In [ ]:
size_of_numpy_array = sys.getsizeof(a_numpy_array)
print(f"A numpy array of a million integers in a list takes {size_of_numpy_array / 1024 / 1024:5.2f} Mb")

## Python is Slow To Run - How Can We Make it Fast? Use Numba!

### Example use case - Approximating PI using the Monte Carlo Method

In [ ]:
from IPython.display import Image
from numba import jit
import random

In [ ]:
Image("Pi_30K.gif")

In [ ]:
def monte_carlo_pi_slow(nsamples: int):
    """
    Approximate PI by taking lots of random samples of points inside a square
    overlapped by a quarter circle and counting the ratio of points that fall
    inside the quarter circle relative to the total number of sample points.
    """
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:
@jit(nopython=True)
def monte_carlo_pi_fast(nsamples: int):
    """
    Approximate PI by taking lots of random samples of points inside a square
    overlapped by a quarter circle and counting the ratio of points that fall
    inside the quarter circle relative to the total number of sample points.
    """
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

### Plain Python

In [ ]:
%%time
monte_carlo_pi_slow(10_000_000)

### First call = compile + runtime

In [ ]:
%%time
monte_carlo_pi_fast(10_000_000)

### Subsequent calls = just runtime

In [ ]:
%%time
monte_carlo_pi_fast(10_000_000)

## Summary

### 1. Use Pandas where you can

Pandas is single-threaded but it is fast. Take advantage of its vectorizing features when doing calculations or adding calculated columns. You should almost never need to loop over rows of a Pandas Dataframe or Series.

### 2. If you start running out of space try the following

1. If your data still fits comfortably on your local SSD or HDD then take advantage of lazy loading packages like MMap, Vaex, Dask, or try a simple database like SQL Lite.
2. If your data is larger than can fit on a local drive you'll need to start using a full database management system.

### 3. If your code runs too slow 

1. Identify whether the code is slow because it is IO bound and is spending a lot of time waiting for data to move around, or whether it is slow because it is CPU bound from crunching so many numbers.
2. If your code is IO bound try the asyncio library and look for async compatible packages, e.g. aiohttp / httpx for asynchronous web requests, aiofiles for asynchronous file reads and writes, and there's many other packages for aynchronous database requests and more.
3. Look for opportunities to parallelize the computation (i.e. across rows of a table, across of loop of calculations, or any other architectures that repeat a lot of calculations independently of each other). If using scikit-learn functions look for the "n_jobs" parameter and set that to -1 to use all your CPU cores (scikit-learn integrates with the joblib package).
4. Try joblib to parallelize work across all your CPU cores, if you have NVidia GPU compute use packages like cudf and cupy (In some cases a small computer with GPU might be faster and cheaper than a cluster of CPU only computers). If you have access to a cluster of computers like a Dask or Spark cluster you can use joblib to parallelize work across a whole cluster of computers and operations with. For the ultimate in parallelizing work look at the [rapids.ai project](https://rapids.ai/) which uses Dask to parallelize across a cluster of computers with GPUs for massively parallel computing.
5. If you have a sequential (non-parallelizable computation) try some of the Just In Time (jit) compiler libraries like Numba. Numba can also compile to GPU CUDA code now too for a relatively easy pathway to GPU accelerated computation.


## Other resources

- I found Michael Kennedy's training courses really great. He has one for [aynchronous programming and parallelizing code](https://training.talkpython.fm/courses/explore_async_python/async-in-python-with-threading-and-multiprocessing) which I learnt a lot from (this is paid).
- Plenty of other free materials - I like [Kaggle Learning](https://www.kaggle.com/learn/overview), [Real Python](https://realpython.com/), and for practising the real basic [Python Tutor](http://www.pythontutor.com/)
- You can practise with most machine learning libraries and GPU accelerated code for free with [Google Colab](https://colab.research.google.com/) and [Kaggle Kernels](https://www.kaggle.com/kernels)
- For nearly free content subscribe to [Humble Bundle](https://humblebundle.com/) they have great and extremely cheap boos and/or video coding and data science bundles almost monthly
- There's plenty of great podcasts if you prefer learning in audio form - happy to give some recommendations if you ask.